In [1]:
# pip install spotipy

In [1]:
# importing the necessary packages
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util
import pandas as pd
import time
import pickle
import multiprocessing
from pandarallel import pandarallel
from simstring.feature_extractor.word_ngram import WordNgramFeatureExtractor
from simstring.measure.jaccard import JaccardMeasure
from simstring.database.mongo import MongoDatabase
from simstring.searcher import Searcher
from simstring.feature_extractor.character_ngram import CharacterNgramFeatureExtractor
from simstring.measure.cosine import CosineMeasure
from simstring.database.dict import DictDatabase
from simstring.searcher import Searcher
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

In [2]:
def time_now():
    now = time.ctime()
    cnvtime = time.strptime(now)
    print(time.strftime("%Y/%m/%d %H:%M", cnvtime))

In [3]:
client_id = 'd6e1e00e6a6a46469a52c941df8a0125'
client_secret = '1a637497f4aa4b7bb47146d8873de94a'
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [3]:
df_hist = pd.read_csv(r'C:\\Users\\daisu\\Desktop\\UChicago\\ML\\Final_Project\\spotify_dataset\\spotify_dataset.csv', on_bad_lines='skip')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [65]:
num_processors = multiprocessing.cpu_count()

In [68]:
pandarallel.initialize(progress_bar=True, nb_workers=num_processors-1, use_memory_fs=False)

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


### Get the User ID and artist name on spotify from the original listening history dataset

In [5]:
len(df_hist)

12891680

In [6]:
user_lst = pd.Series(df_hist['user_id'].unique()).sample(10000).tolist()

In [7]:
df_hist = df_hist[df_hist['user_id'].isin(user_lst)]

In [15]:
df_hist.columns

Index(['user_id', ' "artistname"', ' "trackname"', ' "playlistname"'], dtype='object')

In [16]:
df_hist.columns = ['user_id','artistname','trackname','playlistname']

In [ ]:
### df_hist.to_csv('Spotify_10000usersamples.csv', index=False)

In [52]:
df_hist = pd.read_csv('Spotify_10000usersamples.csv')

In [11]:
len(df_hist['trackname'].unique())

1530367

In [10]:
with open('C:\\Users\\daisu\\Desktop\\UChicago\\ML\\Final_Project\\BackUp_16000\\query_name.p', 'rb') as p:
    query_name = pickle.load(p)

In [12]:
track_unique = pd.Series(df_hist['trackname'].unique()).sample(100000).tolist()

In [13]:
len(df_hist)

8049569

In [14]:
len(df_hist[df_hist['trackname'].isin(track_unique)])

533949

In [24]:
df_temp = df_hist[df_hist['trackname'].isin(track_unique)]

In [25]:
df_temp = df_temp.drop_duplicates(subset='trackname')

In [26]:
tracks = df_temp['trackname'].tolist()
artists = df_temp['artistname'].tolist()

In [27]:
artist_track = []
for i in range(len(tracks)):
    artist_track.append((tracks[i], artists[i]))

In [6]:
artist_track_set = list(set(artist_track))

In [4]:
# with open('artist_track_set.p', 'wb') as p:
#     pickle.dump(artist_track_set, p)
with open('artist_track_set.p', 'rb') as p:
    artist_track_set = pickle.load(p)

In [57]:
backup = 'C:\\Users\\daisu\\Desktop\\UChicago\\ML\\Final_Project\\Roma\\'

In [16]:
%%time
query_name = []
track_name = []
track_id = []
popularity = []
i = 0
time_now()
for track, artist in artist_track_set[51000:70000]:
    try:
        query = sp.search(q= track + ' ' + artist)
        track_name.append(query['tracks']['items'][0]['name'])
        track_id.append(query['tracks']['items'][0]['id'])
        popularity.append(query['tracks']['items'][0]['popularity'])
        query_name.append(track)
    except:
        pass
    if i%1000 == 0:
        print(i)
        time_now()
        df_tracks = pd.DataFrame({
            'query_name':query_name,
            'track_name':track_name, 
            'track_id': track_id,
            'popularity': popularity})
        df_tracks.to_csv('2302_df_tracks.csv')
    time.sleep(.3)
    i += 1

2023/02/27 00:53
0
2023/02/27 00:53


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Capriccio Op. 85: Gut in eure Mäntel gehüllt (Direktor/Graf/Ein Diener/Clairon/Flamand/Olivier) Elisabeth Schwarzkopf/Nicolai Gedda/Dietrich Fischer-Dieskau/Hans Hotter/Christa Ludwig/Rudolf Christ/Anna Moffo/Dermot Troy/Karl Schmitt-Walter/Philharmonia Orchestra/Wolfgang Sawallisch/Eberhard Waechter', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.


1000
2023/02/27 01:02


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'After All the Roads Have Led to Nowhere (The Joke is Always On Us, Sometimes - Live At First Avenue, Minneapolis, Minnesota, 2nd November 1997) - The Joke is Always On Us, Sometimes - Live At First Avenue, Minneapolis, Minnesota, 2nd November 1997 Sugar', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Die Meistersinger von Nürnberg (1999 Digital Remaster), Act One, Scene Three: Fanget an! (Beckmesser/Walther) Theo Adam/Karl Ridderbusch/Eberhard Büchner/Horst Lunow/Sir Geraint Evans/Zoltan Kélémen/Hans-Joachim Rotzsch/Peter Bindszus/Horst Hiestermann/Hermann Christian Polster/Horst Reeh/Siegfried Vogel/René Kollo/Peter Schreier/Helen Donath/Ruth', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.


2000
2023/02/27 01:11
3000
2023/02/27 01:20


...

15000
2023/02/27 03:11
16000
2023/02/27 03:20


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': "Dixit Dominus in D, RV 594: 5. Juravit Dominus - Tu es sacerdos King's College Choir, Cambridge/Academy of Ancient Music/Stephen Cleobury/Benjamin Bayl/Daniel Hyde/Pauline Nobes/Peter Lissauer/Alastair Ross/Helen Gough/Matthew Wadsworth/Sarah Fox/Deborah Norman/Michael Chance/James Gilchrist/Jonathan Le", 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.


17000
2023/02/27 03:29
18000
2023/02/27 03:39


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': "Capriccio Op. 85: Verraten hab' ich meine Gefühle! Elisabeth Schwarzkopf/Nicolai Gedda/Dietrich Fischer-Dieskau/Hans Hotter/Christa Ludwig/Rudolf Christ/Anna Moffo/Dermot Troy/Karl Schmitt-Walter/Philharmonia Orchestra/Wolfgang Sawallisch/Eberhard Waechter", 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.


CPU times: total: 1min 42s
Wall time: 2h 54min 30s


In [17]:
db = DictDatabase(CharacterNgramFeatureExtractor(2))

In [18]:
query_name = []
for track, artist in artist_track_set[50000:70000]:
    query_name.append(track) 

In [19]:
for i in range(len(query_name)):
    try:
        db.add(query_name[i])
    except:
        pass

In [20]:
searcher = Searcher(db, CosineMeasure())

In [21]:
df_tracks['Search_name'] = df_tracks['track_name'].\
apply(lambda x: searcher.search(x,0.8))

In [22]:
df_tracks['Search_name'] = df_tracks['Search_name'].apply(lambda x: x[0] if len(x)>0 else '')

In [23]:
len(df_tracks[df_tracks['Search_name']!=''])

12186

In [24]:
df_tracks = df_tracks[df_tracks['Search_name']!='']

In [25]:
df_tracks.to_csv('2302_tracks_id_name.csv', index=False)

### Get track features

In [15]:
df_tracks = pd.read_csv('2302_tracks_id_name.csv')

In [26]:
track_id_list = df_tracks['track_id'].tolist()

In [27]:
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
track_id = []
uri = []
track_href = []
analysis_url = []
duration_ms = []
time_signature = []
time_now()

for j in range(0, len(track_id_list), 100):
    feature_info = sp.audio_features(track_id_list[j:j+100])
    try:
        for i in range(len(feature_info)):
            danceability.append(feature_info[i]['danceability'])
            energy.append(feature_info[i]['energy'])
            key.append(feature_info[i]['key'])
            loudness.append(feature_info[i]['loudness'])
            mode.append(feature_info[i]['mode'])
            speechiness.append(feature_info[i]['speechiness'])
            acousticness.append(feature_info[i]['acousticness'])
            instrumentalness.append(feature_info[i]['instrumentalness'])
            liveness.append(feature_info[i]['liveness'])
            valence.append(feature_info[i]['valence'])
            tempo.append(feature_info[i]['tempo'])
            track_id.append(feature_info[i]['id'])
            uri.append(feature_info[i]['uri'])
            track_href.append(feature_info[i]['track_href'])
            analysis_url.append(feature_info[i]['analysis_url'])
            duration_ms.append(feature_info[i]['duration_ms'])
            time_signature.append(feature_info[i]['time_signature'])
    except:
        pass
    time.sleep(.3)
    if (j%1000 == 0)&(j!=0):
        df_features = pd.DataFrame({
            'track_id': track_id,
            'danceability':danceability,
            'energy': energy,
            'key': key,
            'loudness': loudness,
            'mode': mode,
            'speechiness': speechiness,
            'acousticness': acousticness,
            'instrumentalness': instrumentalness,
            'liveness': liveness,
            'valence': valence,
            'tempo': tempo,
            'uri': uri,
            'track_href': track_href,
            'analysis_url': analysis_url,
            'duration_ms': duration_ms,
            'time_signature': time_signature
        })
        df_features.to_csv('2302_df_features.csv', index=False)
        time_now()
        print(j)

2023/02/27 03:48
2023/02/27 03:48
1000
2023/02/27 03:49
2000
2023/02/27 03:49
3000
2023/02/27 03:49
4000
2023/02/27 03:49
5000
2023/02/27 03:49
6000
2023/02/27 03:49
7000
2023/02/27 03:49
8000
2023/02/27 03:49
9000
2023/02/27 03:49
10000
2023/02/27 03:49
11000
2023/02/27 03:49
12000


In [45]:
df_features.to_csv('2302_df_features.csv' ,index=False)

In [28]:
df_merge = pd.merge(df_features, df_tracks, on='track_id', how='left')

In [29]:
artist_track_set = pd.DataFrame(artist_track_set, columns=['trackname','artists'])

In [30]:
artist_track_set = artist_track_set.drop_duplicates(subset='trackname')

In [81]:
### artist_track_set.to_csv('artist_track_set.csv', index=False)

In [31]:
df_merge = pd.merge(df_merge, artist_track_set, left_on='Search_name', right_on='trackname', how='left')

In [32]:
df_merge = df_merge.drop_duplicates(subset='track_id').reset_index(drop=True)

In [33]:
#df_merge.to_csv('230227_Tracks_attribute_list.csv', index=False)

In [6]:
df_merge = pd.read_csv('230227_Tracks_attribute_list.csv')

In [4]:
GENIUS_TOKEN='h0fDxuydkla-e6AUiSVuTkSSGfIH7bOmHcVGQF9E1mG4PNpgQfWB-n9GoQxE8xaV'
SCRT='BDcCrDEgOm7YHvfYAQKXTa9qXJ4lKIFaEiFjrZBWJpmItpuDg9ACZtec4rp721xvAuT6k1EtWuPLIuZUzJF4VQ'
CLIENT_ID = '8vuWTETZAE_SmsDB5X6if_pTKcJc_WHAbIjCtiCcce1wkSCn16AxCvabM_s_xtzc'

In [5]:
import requests
from bs4 import BeautifulSoup
import os
import re
import lyricsgenius

In [ ]:
genius = lyricsgenius.Genius(GENIUS_TOKEN)
lyrics_lst = []
artist_lst = []
track_lst = [] 
id_lst = []
non_lst_song = []
non_lst_artist = []
non_lst_id =[]
for i in range(5200, len(df_merge)):
    try:
        artist = genius.search_artist(df_merge['artists'][i], max_songs=1) 
        track = genius.search_song(df_merge['trackname'][i], artist.name)
        if len(track.lyrics)>0:
            lyrics_lst.append(track.lyrics)
            artist_lst.append(df_merge['artists'][i])
            track_lst.append(df_merge['trackname'][i])
            id_lst.append(df_merge['track_id'][i])
    except:
        non_lst_song.append(df_merge['trackname'][i])
        non_lst_artist.append(df_merge['artists'][i])
        non_lst_id.append(df_merge['track_id'][i])
    if i % 100 == 0:
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++',i)
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++')
        df_lyrics = pd.DataFrame(
        {'track_id': id_lst,
        'artist':artist_lst,
        'track':track_lst,
        'lyric':lyrics_lst})
        df_lyrics.to_csv('df_lyrics.csv', index=False)
        
        df_non = pd.DataFrame(
                    {'track_id': non_lst_id,
        'artist':non_lst_artist,
        'track':non_lst_song})
        df_non.to_csv('df_non.csv', index=False)

Searching for songs by Dream Piano...

Changing artist name to 'Panoramic Dreams'
Done. Found 0 songs.
Searching for "Quiet Storm : Part 2" by Panoramic Dreams...
Done.
++++++++++++++++++++++++++++++++++++++++++++++++++++ 5200
++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++
Searching for songs by Over-Reactor...

Song 1: "When Collingwood Choke"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "The Throne" by Over-Reactor...
No results found for: 'The Throne Over-Reactor'
Searching for songs by Thrice...

Song 1: "Black Honey"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Wood And Wire" by Thrice...
Done.
Searching for songs by Doc Watson...

Song 1: "Shady Grove"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Little Stream Of Whisky" by Doc Watson...
Done.
Searching for songs by Jimi Hendrix 